In [2]:
#ORIM Assignment 1:
#Author: Saba Khalid

#import the required libraries:
import pandas as pd
import io
import requests
import numpy as np
from IPython.display import display, HTML
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity ="all"


URL = "https://s3.amazonaws.com/orim-misc-data/assessment/subscribers.csv"
webpage =requests.get(URL).content
df =pd.read_csv(io.StringIO(webpage.decode('utf-8')))
print (df.head())



                 Profile.Id         Domain  Engagement               Signup  \
0  5572fb5592721912198b587b  sbcglobal.net  hardbounce  2015/06/06 09:53:25   
1  5441111b66dfa5ab01b0a000    hotmail.com      optout  2014/10/17 08:52:43   
2  56cde9d9d9cb94486d8b51d7      yahoo.com      optout  2016/02/24 12:35:21   
3  57e68298d9cb942a708b48db      gmail.com     dormant  2016/09/24 09:41:44   
4  5498bd77f641910f100e23fb  sbcglobal.net      optout  2014/12/22 19:55:19   

   Opens  Clicks            Last.Open           Last.Click  \
0     51      11  2016/01/20 00:35:25  2015/08/12 19:50:19   
1      2       0  2014/11/21 07:37:15                  NaN   
2      0       0                  NaN                  NaN   
3      0       0                  NaN                  NaN   
4     30       1  2015/01/06 21:10:26  2015/01/01 21:21:20   

           Optout.Time        Geolocation.City  \
0                  NaN  El Dorado Hills, CA US   
1  2014/11/21 07:45:29                     NaN   
2 

In [3]:
#create a copy of the the two columns needed into a new Dataframe df1
df1 = df[['referral_source','Signup']].copy()
print (df1.head(10))

  referral_source               Signup
0             DMi  2015/06/06 09:53:25
1        Untagged  2014/10/17 08:52:43
2             DMi  2016/02/24 12:35:21
3        Facebook  2016/09/24 09:41:44
4             NaN  2014/12/22 19:55:19
5      In Book Ad  2016/10/07 21:45:30
6      LiveIntent  2016/05/20 10:42:54
7             DMi  2015/09/14 12:53:41
8        Untagged  2014/10/01 19:15:03
9        Facebook  2014/11/23 15:16:35


In [4]:
#now that a new dataframe has been created we can preprocess it for our usage
#convert the data type of the Date from object to datetime64: 
df1['Signup']= pd.to_datetime(df1['Signup'], format ='%Y-%m-%d')
print (df1.head(10))
print (df1.dtypes)

  referral_source              Signup
0             DMi 2015-06-06 09:53:25
1        Untagged 2014-10-17 08:52:43
2             DMi 2016-02-24 12:35:21
3        Facebook 2016-09-24 09:41:44
4             NaN 2014-12-22 19:55:19
5      In Book Ad 2016-10-07 21:45:30
6      LiveIntent 2016-05-20 10:42:54
7             DMi 2015-09-14 12:53:41
8        Untagged 2014-10-01 19:15:03
9        Facebook 2014-11-23 15:16:35
referral_source            object
Signup             datetime64[ns]
dtype: object


In [5]:
#Gather info about the Dataframe:
print (df1.describe())

       referral_source               Signup
count           630008               705144
unique              75               593820
top                DMi  2017-04-07 15:45:49
freq            243435                 1443
first              NaN  2014-06-16 15:14:30
last               NaN  2017-04-20 00:57:58


In [6]:
#Rename the columns 
df1.rename(columns={'referral_source':'Source','Signup':'Signup_Date'}, inplace =True)
df1['Signup_Date']=df1['Signup_Date'].dt.normalize()         #normalize the column to remove the timestamp from displaying without changing the dtype
df1.head()


,Source,Signup_Date
0,DMi,2015-06-06
1,Untagged,2014-10-17
2,DMi,2016-02-24
3,Facebook,2016-09-24
4,NaN,2014-12-22


In [7]:
#change the NaN in the Source to organic source and count subscribers for that with the rest of the data:

df1['Source']=df1['Source'].replace(np.nan,'Organic Source')
df1.head()


,Source,Signup_Date
0,DMi,2015-06-06
1,Untagged,2014-10-17
2,DMi,2016-02-24
3,Facebook,2016-09-24
4,Organic Source,2014-12-22


In [8]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th{ border: 1px black solid !important;
    color: black !important;
    }
</style>


In [9]:
#Now we group the Source by Year and Month using dt.year and dt.month (datetime64 dtype)
#Count the number of subscribers for each given Source, Year and Month:
df2=df1['Source'].groupby([df1['Source'],df1['Signup_Date'].dt.year.rename('Year'),df1['Signup_Date'].dt.month.rename('Month')]).agg({'count'})
#print (df2.head(50))
#print (df2)
display(df2)

count
Source     Year Month       
AdRoll     2014 9          1
                11         1
                12         3
BookPage   2014 7         17
                8         16
                9          7
                10        13
                11         6
                12         9
           2015 1          6
                2          4
                3          3
                4          1
                5          1
                6          2
                7          3
                8          1
                9          3
                10         2
                11         6
                12         1
           2016 1          5
                2          9
                3          9
                4          7
                5          8
                6          5
                7          6
                8          4
                9          1
...                      ...
torsweeps  2015 11        75
                12        37
           2016 1         95
                2         81
                3         76
                4         79
                5         92
                6         55
                7        151
                8         63
                9         33
                10        48
                11       167
                12       168
           2017 1        150
                2      10460
                3       3414
                4       4625
twitter    2016 7          1
worth      2015 8          1
           2016 3          1
                5          2
worthbooks 2014 12         1
           2016 3          1
                7          1
                10         1
           2017 1          3
                2          6
                3          1
                4          6

[914 rows x 1 columns]

In [10]:
#part 3 of Assignment 1:

df_genre=df['ebb_preferences']
print (df_genre.dtypes)
print (df_genre.head(10))








object
0    ["bestsellers","mysteries_thrillers_true_crime...
1                                                  NaN
2                                                  NaN
3                    ["bestsellers","book_club_picks"]
4                                                  NaN
5    ["historical_fiction","literary_fiction","myst...
6    ["childrens","historical_fiction","literary_fi...
7                           ["biographies_and_memoir"]
8                                                  NaN
9                                                  NaN
Name: ebb_preferences, dtype: object


In [11]:
#df_genre=df_genre.replace(np.nan,0)
#df_genre.fillna(0, inplace =True)

#first count the number of preferences listed within each cell
#Change the NaN to the value 0
#Change the data type from object to int

df['New_column']= df_genre.str.split(",").str.len()
print (df.dtypes)
df['New_column']=df['New_column'].replace(np.nan,0)
df['New_column']=df['New_column'].astype(int)
print (df['New_column'].head(10))
df['New_column']
print (df.dtypes)


Profile.Id              object
Domain                  object
Engagement              object
Signup                  object
Opens                    int64
Clicks                   int64
Last.Open               object
Last.Click              object
Optout.Time             object
Geolocation.City        object
Geolocation.State       object
Geolocation.Country     object
Geolocation.Zip         object
Lifetime.Message         int64
Top.Device              object
referral_source         object
first_name              object
last_name               object
age                    float64
gender                  object
ebb_preferences         object
New_column             float64
dtype: object
0     5
1     0
2     0
3     2
4     0
5     9
6    12
7     1
8     0
9     0
Name: New_column, dtype: int64
Profile.Id              object
Domain                  object
Engagement              object
Signup                  object
Opens                    int64
Clicks                   int64
Last.Op

In [11]:
#Count the max number of preferences within this column:
max_value=df['New_column'].max()
print (max_value)


16


In [12]:
#Replace the 0 values with the maximum number of preferences:#(assuming that max contains all max preferences)
#(the Nan values represent all preferences)
df['New_column']=np.where(df['New_column'] ==0,max_value,df['New_column'])
display (df['New_column'])

0          5
1         16
2         16
3          2
4         16
5          9
6         12
7          1
8         16
9         16
10        10
11         8
12        14
13         5
14        16
15         8
16        10
17         9
18         5
19        16
20        14
21         6
22        13
23        16
24        14
25        14
26        16
27        16
28        16
29         8
          ..
705114    16
705115     4
705116     5
705117     5
705118     3
705119     3
705120     7
705121    16
705122     7
705123     1
705124     8
705125     8
705126     1
705127     1
705128     9
705129     4
705130    16
705131     2
705132     1
705133    14
705134     5
705135    13
705136     7
705137    10
705138     4
705139     1
705140     4
705141     5
705142    13
705143     2
Name: New_column, Length: 705144, dtype: int64

In [13]:
#preprocess the data before we can make the binary columns for the column ebb_preferences:

df_preferences =df['ebb_preferences']

print (df_preferences.head(5))

0    ["bestsellers","mysteries_thrillers_true_crime...
1                                                  NaN
2                                                  NaN
3                    ["bestsellers","book_club_picks"]
4                                                  NaN
Name: ebb_preferences, dtype: object


In [14]:
#replace the missing value with "All"
df_preferences=df_preferences.replace(np.nan,'["All Genres"]')
df_preferences.columns =["ebb_preferences"]
print (df_preferences.head(5))
print (df_preferences.dtype)





0    ["bestsellers","mysteries_thrillers_true_crime...
1                                       ["All Genres"]
2                                       ["All Genres"]
3                    ["bestsellers","book_club_picks"]
4                                       ["All Genres"]
Name: ebb_preferences, dtype: object
object


In [15]:
#DATA CLEANING:
#convert to lower case:
df_preferences=df_preferences.str.lower()
#df_pre

#nltk to remove stopwords from the DataFrame:
from nltk.corpus import stopwords
stop_words =stopwords.words('english')

df_preferences=df_preferences.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))







In [16]:
##used this to convert everthing as strings
import json
df_preferences.to_json()
#df_pre


#used this to change the strings to list. 
L_preferences=df_preferences.apply(lambda x: x[1:-1].split(','))
#display (L2)

#print (L2[2])

#count the longest list and elements within that list for EXPLORATION:
maxlist =max(L_preferences, key =len)
maxlength = max(map(len, L_preferences))

print (maxlist, maxlength)



['"bestsellers"', '"book_club_picks"', '"biographies_and_memoir"', '"childrens"', '"general_nonfiction"', '"historical_fiction"', '"history"', '"lifestyle_and_cooking"', '"literary_fiction"', '"mysteries_thrillers_true_crime"', '"poetry"', '"romance"', '"science_fiction_fantasy"', '"young_adult"', '"bestsellers"', '"book_club_picks"'] 16


In [17]:
#cleaning lists in list from punctuation and removing words with len<3:
import string
cleaned_list=[]

for x in L_preferences:
    #print (x)
    x=[''.join(c for c in s if c not in string.punctuation) for s in x]
    
    #removing words that are less than 3:
    x=[word for word in x if len(word) >3]
    cleaned_list.append(x)
cleaned_list[:20]



    
        

[['bestsellers',
  'mysteriesthrillerstruecrime',
  'sciencefictionfantasy',
  'literaryfiction',
  'generalnonfiction'],
 ['all genres'],
 ['all genres'],
 ['bestsellers', 'bookclubpicks'],
 ['all genres'],
 ['historicalfiction',
  'literaryfiction',
  'mysteriesthrillerstruecrime',
  'romance',
  'youngadult',
  'biographiesandmemoir',
  'lifestyleandcooking',
  'bestsellers',
  'bookclubpicks'],
 ['childrens',
  'historicalfiction',
  'literaryfiction',
  'sciencefictionfantasy',
  'youngadult',
  'biographiesandmemoir',
  'generalnonfiction',
  'history',
  'lifestyleandcooking',
  'poetry',
  'bestsellers',
  'bookclubpicks'],
 ['biographiesandmemoir'],
 ['all genres'],
 ['all genres'],
 ['historicalfiction',
  'literaryfiction',
  'mysteriesthrillerstruecrime',
  'sciencefictionfantasy',
  'youngadult',
  'generalnonfiction',
  'history',
  'lifestyleandcooking',
  'bestsellers',
  'bookclubpicks'],
 ['historicalfiction',
  'literaryfiction',
  'mysteriesthrillerstruecrime',
  'b

In [18]:
##remove repeated Genres from the same lists:
pref_list=[]
for l in cleaned_list:
    new_l=list(set(l))
    pref_list.append(new_l)
print (pref_list[:50])



[['generalnonfiction', 'mysteriesthrillerstruecrime', 'bestsellers', 'literaryfiction', 'sciencefictionfantasy'], ['all genres'], ['all genres'], ['bestsellers', 'bookclubpicks'], ['all genres'], ['romance', 'bookclubpicks', 'lifestyleandcooking', 'mysteriesthrillerstruecrime', 'bestsellers', 'biographiesandmemoir', 'historicalfiction', 'youngadult', 'literaryfiction'], ['bookclubpicks', 'lifestyleandcooking', 'poetry', 'generalnonfiction', 'history', 'childrens', 'bestsellers', 'biographiesandmemoir', 'historicalfiction', 'youngadult', 'literaryfiction', 'sciencefictionfantasy'], ['biographiesandmemoir'], ['all genres'], ['all genres'], ['bookclubpicks', 'lifestyleandcooking', 'generalnonfiction', 'mysteriesthrillerstruecrime', 'history', 'bestsellers', 'historicalfiction', 'youngadult', 'literaryfiction', 'sciencefictionfantasy'], ['bookclubpicks', 'generalnonfiction', 'mysteriesthrillerstruecrime', 'bestsellers', 'biographiesandmemoir', 'historicalfiction', 'literaryfiction', 'histo

In [19]:
#change the list into Series in pandas:
series_pref =pd.Series(pref_list)
series_pref

0         [generalnonfiction, mysteriesthrillerstruecrim...
1                                              [all genres]
2                                              [all genres]
3                              [bestsellers, bookclubpicks]
4                                              [all genres]
5         [romance, bookclubpicks, lifestyleandcooking, ...
6         [bookclubpicks, lifestyleandcooking, poetry, g...
7                                    [biographiesandmemoir]
8                                              [all genres]
9                                              [all genres]
10        [bookclubpicks, lifestyleandcooking, generalno...
11        [bookclubpicks, generalnonfiction, mysteriesth...
12        [romance, bookclubpicks, lifestyleandcooking, ...
13        [romance, mysteriesthrillerstruecrime, bestsel...
14                                             [all genres]
15        [romance, lifestyleandcooking, poetry, mysteri...
16        [bookclubpicks, poetry, genera

In [20]:
#sklearn gives binary columns for 188 unique preferences for L2:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
new_df = df.join(pd.DataFrame(mlb.fit_transform(series_pref),columns=mlb.classes_))
display (new_df)
print (new_df.info())

#extract the columns of this dataframe in an array to check for column headings
dummie_names2 =new_df.columns.values
dummie_names2

,Profile.Id,Domain,Engagement,Signup,Opens,Clicks,Last.Open,Last.Click,Optout.Time,Geolocation.City,...,young,young adult,younga,youngad,youngadu,youngadul,youngadult,ysteries,ysteriesthrillerstruecrim,ysteriesthrillerstruecrime
0,5572fb5592721912198b587b,sbcglobal.net,hardbounce,2015/06/06 09:53:25,51,11,2016/01/20 00:35:25,2015/08/12 19:50:19,NaN,"El Dorado Hills, CA US",...,0,0,0,0,0,0,0,0,0,0
1,5441111b66dfa5ab01b0a000,hotmail.com,optout,2014/10/17 08:52:43,2,0,2014/11/21 07:37:15,NaN,2014/11/21 07:45:29,NaN,...,0,0,0,0,0,0,0,0,0,0
2,56cde9d9d9cb94486d8b51d7,yahoo.com,optout,2016/02/24 12:35:21,0,0,NaN,NaN,2016/03/02 19:45:14,NaN,...,0,0,0,0,0,0,0,0,0,0
3,57e68298d9cb942a708b48db,gmail.com,dormant,2016/09/24 09:41:44,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5498bd77f641910f100e23fb,sbcglobal.net,optout,2014/12/22 19:55:19,30,1,2015/01/06 21:10:26,2015/01/01 21:21:20,2015/01/06 21:11:33,NaN,...,0,0,0,0,0,0,0,0,0,0
5,57f84fbad9cb94f1778b50d7,hotmail.com,passive,2016/10/07 21:45:30,54,29,2017/04/17 15:29:19,2017/04/06 10:34:26,NaN,"Birmingham, MI US",...,0,0,0,0,0,0,1,0,0,0
6,573f226e72fbf2aa4c8b4d15,yahoo.com,optout,2016/05/20 10:42:54,16,0,2016/05/25 20:24:56,NaN,2016/05/25 20:26:02,NaN,...,0,0,0,0,0,0,1,0,0,0
7,55f6fb9527fdb0b27e8b4827,gmail.com,disengaged,2015/09/14 12:53:41,16,0,2017/02/05 20:51:11,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8,542c8af71939acb4378b4583,hotmsil.com,hardbounce,2014/10/01 19:15:03,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
9,547240a366dfa5cb6616a667,yahoo.com,hardbounce,2014/11/23 15:16:35,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705144 entries, 0 to 705143
Columns: 141 entries, Profile.Id to ysteriesthrillerstruecrime
dtypes: float64(1), int64(123), object(17)
memory usage: 758.6+ MB
None


array(['Profile.Id', 'Domain', 'Engagement', 'Signup', 'Opens', 'Clicks',
       'Last.Open', 'Last.Click', 'Optout.Time', 'Geolocation.City',
       'Geolocation.State', 'Geolocation.Country', 'Geolocation.Zip',
       'Lifetime.Message', 'Top.Device', 'referral_source', 'first_name',
       'last_name', 'age', 'gender', 'ebb_preferences', 'New_column',
       '  true crime', ' thrillers', ' thrillers  true crime',
       'all genres', 'bestsellers', 'biographiesandmemoi',
       'biographiesandmemoir', 'biography memoir', 'bookclubpicks',
       'chil', 'child', 'childr', 'childre', 'children', 'childrens',
       'ciencefictio', 'ciencefictionfantas', 'estseller', 'estsellers',
       'gene', 'gener', 'genera', 'general', 'general nonfiction',
       'generaln', 'generalno', 'generalnon', 'generalnonf',
       'generalnonfi', 'generalnonfic', 'generalnonfict',
       'generalnonficti', 'generalnonfictio', 'generalnonfiction',
       'hildren', 'hist', 'histor', 'historical fiction',

In [21]:
#second method in Pandas using lambda:

dummies2=series_pref.apply(lambda x: pd.Series([1]* len(x), index=x)).fillna(0,)
dummies2

,generalnonfiction,mysteriesthrillerstruecrime,bestsellers,literaryfiction,sciencefictionfantasy,all genres,bookclubpicks,romance,lifestyleandcooking,biographiesandmemoir,...,hildren,generalnonfict,literary fictio,thrillers true crime,ysteries,omeebbpre,lifestyle cooking,oungadul,omance,istoricalfiction
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
